In [1]:
import os
from pathlib import Path
import pickle
from dataclasses import dataclass, field
from typing import Any, Dict, List
from datasets import load_dataset
from tqdm import tqdm
import random
from common.metadata_db import MetadataDB
from common.tensor_storage import TensorStorage
import numpy as p
import pandas as pd


/home/alexserra98/miniconda3/envs/mcqa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
Features

datasets.features.features.Features

In [9]:
import random
s = {"A","B","C","D"}-{"B"}
sample = random.choice(list(s))
sample


'C'

In [3]:
from datasets import load_dataset
from datasets import load_dataset,Features,Value,Sequence,Array2D,Array3D,Array4D,Array5D
context_feat = Features({'id':  Value(dtype='string', id=None),
 'question':  Value(dtype='string', id=None),
 'question_concept':  Value(dtype='string', id=None),
 'choices': Sequence(feature={'label': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}, length=-1, id=None),
 'answerKey': Value(dtype='string', id=None)})
dataset = load_dataset('tau/commonsense_qa')


Generating test split: 100%|██████████| 1140/1140 [00:00<00:00, 105171.27 examples/s]


In [13]:
from datasets import load_dataset
dataset = load_dataset('Stevross/mmlu','professional_law',trust_remote_code=True)

Generating dev split: 100%|██████████| 5/5 [00:00<00:00, 1395.40 examples/s]


In [26]:
dataset["test"][0]

{'id': '90b30172e645ff91f7171a048582eb8b',
 'question': 'The townhouse was a hard sell for the realtor, it was right next to a high rise what?',
 'question_concept': 'townhouse',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['suburban development',
   'apartment building',
   'bus stop',
   'michigan',
   'suburbs']},
 'answerKey': ''}

In [22]:
@dataclass
class RequestInstance():
    prompt: str
    letter_gold: str
    token_gold: int = None

In [2]:
dataset = load_dataset("openbookqa")

In [6]:
type(dataset["train"])

datasets.arrow_dataset.Dataset

In [39]:
dataset["test"][0]["choices"]["label"]


['A', 'B', 'C', 'D']

In [29]:
def construct_question(row,shot=False):
    prompt = f'Question: {row["question_stem"]}\n'
    for letter, choice in zip(row["choices"]["label"],row["choices"]["text"]):
        prompt += f'{letter}. {choice}\n'
    prompt += f'Answer: {row["choices"]["text"][row["choices"]["label"].index(row["answerKey"])]}\n\n' if shot else  f'Answer:' 
    return prompt 
def construct_request_instance():
    """
    Construct the request instances for the scenario
    """

    output_mapping = dataset["test"][0]["choices"]["label"]
    
    ri = []

    dataset_test = dataset["test"].select(range(5)) \
                                if 5 != -1 else dataset["test"]
    for row in tqdm(dataset_test, desc="Constructing Prompts"):
        prompt = f'The following are multiple choice questions (with answers) about {"commonsense"}.\n\n'
        for i in range(5):
            random_row = dataset["validation"][i]
            prompt += construct_question(random_row,shot=True)
        prompt += construct_question(row)
        ri.append(RequestInstance(prompt, row["answerKey"]))
    return ri, output_mapping

In [34]:
print(construct_request_instance()[0][0].prompt)

Constructing Prompts: 100%|██████████| 5/5 [00:00<00:00, 1070.14it/s]

The following are multiple choice questions (with answers) about commonsense.

Question: Frilled sharks and angler fish live far beneath the surface of the ocean, which is why they are known as
A. Deep sea animals
B. fish
C. Long Sea Fish
D. Far Sea Animals
Answer: Deep sea animals

Question: Gas can fill any container it is given, and liquid
A. is standard weight and size
B. is the opposite of variable
C. only needs a few
D. uses what it needs
Answer: uses what it needs

Question: When birds migrate south for the winter, they do it because
A. they are genetically called to
B. their children ask for them to
C. it is important to their happiness
D. they decide to each year
Answer: they are genetically called to

Question: If a person walks in the opposite direction of a compass arrow they are walking
A. west
B. north
C. east
D. south
Answer: south

Question: An example of lots kinetic energy would be
A. Drinking a cold glass of water
B. A snail moving across the sidewalk
C. sitting with

In [20]:
# Features.from_dict(
# {'id': '075e483d21c29a511267ef62bedc0461',
#  'question': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?',
#  'question_concept': 'punishing',
#  'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
#   'text': ['ignore', 'enforce', 'authoritarian', 'yell at', 'avoid']},
#  'answerKey': 'A'}
# )
Features({'id':  Value(dtype='string', id=None),
 'question':  Value(dtype='string', id=None),
 'question_concept':  Value(dtype='string', id=None),
 'choices': Sequence(feature={'label': Value(dtype='string', id=None), 'text': Value(dtype='int32', id=None)}, length=-1, id=None),
 'answerKey': Value(dtype='string', id=None)})

{'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'question_concept': Value(dtype='string', id=None),
 'choices': Sequence(feature={'label': Value(dtype='string', id=None), 'text': Value(dtype='int32', id=None)}, length=-1, id=None),
 'answerKey': Value(dtype='string', id=None)}